In [1]:
import os, glob
import numpy as np
import pandas as pd
import math
import pickle
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import stats
from statsmodels.stats.weightstats import ztest
from IPython.display import display, HTML

In [2]:
base_dir = "/data/private/pdutta/DNABERT_data/Genetic_variant/Brain/Acceptor"

In [3]:
df_acceptor = pd.read_csv("/data/private/pdutta/DNABERT_data/Splice_sites/Acceptor_raw_files/Unique_Acceptor_RAW_Combined_BED.tsv", sep="\t")
df_acceptor

,chr_name,start,end,strand,transcript_id,exon_id,transcript_type
0,chr1,12572,12651,+,"ENST00000450305.2, ENST00000456328.2","ENSE00001758273.2, ENSE00003582793.1","transcribed_unprocessed_pseudogene, processed_..."
1,chr1,12934,13013,+,ENST00000450305.2,ENSE00001799933.2,transcribed_unprocessed_pseudogene
2,chr1,13180,13259,+,"ENST00000450305.2, ENST00000456328.2","ENSE00001746346.2, ENSE00002312635.1","transcribed_unprocessed_pseudogene, processed_..."
3,chr1,13412,13491,+,ENST00000450305.2,ENSE00001863096.1,transcribed_unprocessed_pseudogene
4,chr1,14999,15078,-,ENST00000488147.1,ENSE00001935574.1,unprocessed_pseudogene
...,...,...,...,...,...,...,...
295391,chrY,57211720,57211799,+,"ENST00000340131.12_PAR_Y, ENST00000359512.8_PA...","ENSE00001956598.1, ENSE00001416295.3, ENSE0000...","retained_intron, protein_coding, processed_tra..."
295392,chrY,57213086,57213165,-,ENST00000507418.6_PAR_Y,ENSE00002023900.1,unprocessed_pseudogene
295393,chrY,57213318,57213397,-,ENST00000507418.6_PAR_Y,ENSE00002036959.1,unprocessed_pseudogene
295394,chrY,57213563,57213642,-,ENST00000507418.6_PAR_Y,ENSE00002021169.1,unprocessed_pseudogene


In [4]:
with open("/data/projects/GDC_Cancer_Wise/Brain/Data/raw_acceptor_vcf_data.pkl", "rb") as file:
    loaded_dictionary = pickle.load(file)

In [5]:
def log_oods_ratio(p1,p2):
    term1 = math.log2(p1/(1-p1))
    term2 = math.log2(p2/(1-p2))
    return (term1-term2)
def calculate_diff_probab(p1,p2):
    return((p2-p1)*max(p1,p2))

In [7]:
patient_dfs= []
patient_id = []
for dirpath, dirnames, filenames in os.walk(base_dir):
    #print(dirpath, dirnames, filenames)
    for dir_name in dirnames:
        dir_path = dirpath +"/" + dir_name
        print(dir_path)
        patient_id.append(dir_name)
        df_temp = loaded_dictionary[dir_name]
        concat_probab = np.load(dir_path+"/Prediction_result/pred_results.npy")
        print(len(concat_probab))
        Reference_probab = concat_probab[::2]
        Alternative_probab = concat_probab[1::2]
        df_temp['Ref_probab']=Reference_probab
        df_temp['Alt_probab']=Alternative_probab
        df_temp['ScoreChange'] = df_temp.apply(lambda row: calculate_diff_probab(row['Ref_probab'], row['Alt_probab']), axis=1)
        df_temp['LogOddRatio'] = df_temp.apply(lambda row: log_oods_ratio(row['Ref_probab'], row['Alt_probab']), axis=1)
        #display(df_temp)
        patient_dfs.append(df_temp)
        #input()
    break

/data/private/pdutta/DNABERT_data/Genetic_variant/Brain/Acceptor/722d3df5-7254-44f0-bbe0-cf39dbd57866
56896
/data/private/pdutta/DNABERT_data/Genetic_variant/Brain/Acceptor/ddd29528-58ad-461f-8509-4e42f71b49fb
28548
/data/private/pdutta/DNABERT_data/Genetic_variant/Brain/Acceptor/9593d4de-0dab-4d1e-b86c-af91cac938e6
38748
/data/private/pdutta/DNABERT_data/Genetic_variant/Brain/Acceptor/e849abef-b55e-4124-b760-09101a7c4390
53054
/data/private/pdutta/DNABERT_data/Genetic_variant/Brain/Acceptor/0c700657-b0b8-4361-9f89-b89a6c7b262d
48918
/data/private/pdutta/DNABERT_data/Genetic_variant/Brain/Acceptor/a60a9995-6a02-4035-9c3e-2fee0e6e87c5
38254
/data/private/pdutta/DNABERT_data/Genetic_variant/Brain/Acceptor/4bc99b08-8b67-443e-88a4-03edef39d433
52098
/data/private/pdutta/DNABERT_data/Genetic_variant/Brain/Acceptor/0fce50b4-028d-4b54-aa49-3a8e28a7f7c6
51864
/data/private/pdutta/DNABERT_data/Genetic_variant/Brain/Acceptor/8f36ad24-c425-4474-9d03-a5198e2e4901
51174
/data/private/pdutta/DNABERT

In [8]:
# Concatenate the patient dataframes and drop duplicates to get all unique regions across the patients
unique_patient_regions = pd.concat(patient_dfs).drop_duplicates(subset=['chr', 'Transcript_ID', 'Acceptor_start', 'Acceptor_end'])
unique_patient_regions

,index,chr,strand,Transcript_ID,Acceptor_start,Acceptor_end,varinat_start,variant_end,ref_neucleotide,alternative_neucleotide,reference_seq,alt_seq,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
0,0,chr1,+,ENST00000450305.2,13412,13491,13417,13418,G,A,ACCCCGAGATCACATTTCTCACTGCCTTTTGTCTGCCCAGTTTCAC...,ACCCCAAGATCACATTTCTCACTGCCTTTTGTCTGCCCAGTTTCAC...,9.999686e-01,0.999954,-1.466228e-05,0.553559
1,1,chr1,+,ENST00000624431.2,183091,183170,183143,183144,G,A,TGTGTGGGCCCACCGGCCCCAGGCTCCTGTCTCCCCCCAGGTGTGT...,TGTGTGGGCCCACCGGCCCCAGGCTCCTGTCTCCCCCCAGGTGTGT...,9.999986e-01,0.999998,-4.768365e-07,0.415038
2,2,chr1,-,ENST00000623083.4,187248,187327,187301,187302,A,G,GGGGCGGTGGGGGTGGTGTTAGTACCCCATCTTGTAGGTCTGAAAC...,GGGGCGGTGGGGGTGGTGTTAGTACCCCATCTTGTAGGTCTGAAAC...,1.336061e-06,0.000002,7.127009e-13,-0.384897
3,3,chr1,+,ENST00000669836.1,268081,268160,268129,268130,G,T,CTACAGCAGATTCACTCTGTTCTGTTTCATTGTTGTTTAGTTTGCG...,CTACAGCAGATTCACTCTGTTCTGTTTCATTGTTGTTTAGTTTGCG...,8.743904e-01,0.995606,1.206832e-01,-5.024607
4,4,chr1,-,ENST00000424587.7,268777,268856,268832,268833,T,A,ATAGGAGTTCTGCATTGGTTTGGTCTGCATTGGTTTGGTCTGGAAG...,ATAGGAGTTCTGCATTGGTTTGGTCTGCATTGGTTTGGTCTGGAAG...,2.960983e-06,0.000002,-3.814653e-12,0.823919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16470,16470,chr22,-,"ENST00000360806.9, ENST00000450651.5, ENST0000...",21484095,21484174,21484115,21484116,A,C,TCTCCTCTACTAACTGCTCCAGGAGGTCGCCGATGTCAGCTGCCAA...,TCTCCTCTACTAACTGCTCCCGGAGGTCGCCGATGTCAGCTGCCAA...,3.908874e-03,0.009965,6.034855e-05,-1.358908
19724,19724,chr5,+,"ENST00000264668.6, ENST00000440940.7, ENST0000...",7896823,7896902,7896843,7896855,CTTTTTTTTTTT,CTTTTTTTTTTTT,CTTTATATCACACACCTAAACTTTTTTTTTTTCCACTTAGAGAGAA...,CTTTATATCACACACCTAAACTTTTTTTTTTTTCCACTTAGAGAGA...,9.999940e-01,0.999994,2.384172e-07,-0.058894
24792,24792,chr9,+,"ENST00000382099.3, ENST00000382100.8, ENST0000...",2647433,2647512,2647433,2647434,C,T,CTAAACCACTGAGGCTTATTTCTCATTTAATTTTTCACAGCTTTGT...,TTAAACCACTGAGGCTTATTTCTCATTTAATTTTTCACAGCTTTGT...,9.999925e-01,0.999991,-1.788126e-06,0.308125
25149,25149,chr9,+,"ENST00000431724.2, ENST00000637846.1",85840709,85840788,85840736,85840746,ATTTTTTTTT,ATTTTTTT,AACACTAACAAATAAGTTAACTTTCTGATTTTTTTTTAAGATGTAG...,AACACTAACAAATAAGTTAACTTTCTGATTTTTTTAAGATGTAGAA...,9.999976e-01,0.999986,-1.192090e-05,2.584980


In [9]:
unique_patient_regions[unique_patient_regions['Acceptor_start']==28452437]

,index,chr,strand,Transcript_ID,Acceptor_start,Acceptor_end,varinat_start,variant_end,ref_neucleotide,alternative_neucleotide,reference_seq,alt_seq,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
2882,2882,chr12,+,"ENST00000381259.5, ENST00000535212.1, ENST0000...",28452437,28452516,28452492,28452493,G,A,CAGTCTTTCAAATTTGTTCTGGTCTTTATTTATTTTGAAGCTTGAA...,CAGTCTTTCAAATTTGTTCTGGTCTTTATTTATTTTGAAGCTTGAA...,0.999977,0.999983,6.437192e-06,-0.462190
23430,23430,chr8,-,"ENST00000346498.6, ENST00000380254.7, ENST0000...",28452437,28452516,28452513,28452516,TAA,TAAA,TTGAACGTCAGCGATTACAAATCCATCCTTTGGGGGTGTCTAGAAG...,TTGAACGTCAGCGATTACAAATCCATCCTTTGGGGGTGTCTAGAAG...,0.000150,0.000142,-1.259058e-09,0.083062


In [10]:
# Step 3: Create a new dataframe with three rows for each patient, with columns as the acceptor coordinates
# Initialize the dictionaries
data_ref, data_alt, data_log_odd, data_score_change = {}, {}, {}, {}

# Loop over each region in the unique_patient_regions
for _, region in unique_patient_regions.iterrows():
    key = f'{region.chr}_{region.Acceptor_start}_{region.Acceptor_end}_{region.varinat_start}_{region.variant_end}'
    data_ref[key] = []
    data_alt[key] = []
    data_log_odd[key] = []

# Loop through each patient and for each region, get the Alt_probab score for that region for that patient
for df in patient_dfs:
    print(df.shape)
    #print(df.columns)
    for _, region in unique_patient_regions.iterrows():
        for data, score_column in zip([data_ref, data_alt, data_log_odd, data_score_change], ['Ref_probab', 'Alt_probab', 'LogOddRatio', 'ScoreChange']):
            score_series = df.loc[(df['chr'] == region.chr) & (df['Acceptor_start'] == region.Acceptor_start) & (df['varinat_start'] == region.varinat_start) , score_column]
            if score_series.empty:
                score = np.nan
            else:
                score = score_series.max()
                #print(score_series , score)
                #input()
            data[f'{region.chr}_{region.Acceptor_start}_{region.Acceptor_end}_{region.varinat_start}_{region.variant_end}'].append(score)

(28448, 16)


KeyboardInterrupt: 

In [ ]:
result_df_ref = pd.DataFrame(data_ref)
result_df_ref.insert(0, "Patient_ID", patient_id)
result_df_ref

In [ ]:
result_df_alt = pd.DataFrame(data_alt)
result_df_alt.insert(0, "Patient_ID", patient_id)
result_df_alt

In [ ]:
result_df_log_odd = pd.DataFrame(data_log_odd)
result_df_log_odd.insert(0, "Patient_ID", patient_id)
result_df_log_odd

In [ ]:
result_df_ref.to_csv("/data/projects/GDC_Cancer_Wise/Brain/Data/DNABERT_result/Acceptor/df_ref_score.tsv", sep="\t", index=False)

In [ ]:
result_df_alt.to_csv("/data/projects/GDC_Cancer_Wise/Brain/Data/DNABERT_result/Acceptor/df_alt_score.tsv", sep="\t", index=False)

In [ ]:
result_df_log_odd.to_csv("/data/projects/GDC_Cancer_Wise/Brain/Data/DNABERT_result/Acceptor/df_log_odd_score.tsv", sep="\t", index=False)